# 姿态估计
1. 已知相机矩阵，失真系数等。
2. 给定图案图像，我们可以利用以上信息来计算其姿势或物体在空间中的位置，例如其旋转方式， 对于平面物体，我们可以假设Z = 0，这样，问题就变成了如何将相机放置在空间中以查看图案图像。 
3. 因此，如果我们知道对象在空间中的位置，则可以在其中绘制一些2D图以模拟3D效果。 让我们看看如何做。


## 姿态估计的步骤
1. **加载相机参数**：将之前获取的相机矩阵和失真系数加载到代码中。
2. **检测图案**：在图像中检测出用于姿态估计的特定图案，例如棋盘格。
3. **查找图案角点**：使用图案检测算法（如OpenCV的findChessboardCorners函数）找到图案的角点。
4. **计算姿态**：利用相机矩阵、失真系数和图案的角点，使用姿态估计算法（例如棋盘格的solvePnP函数）计算物体在空间中的位置和方向。
5. **绘制空间图像**：通过得到的物体位置和方向，可以在图像中绘制出对应的3D效果，模拟物体在空间中的姿态。这可以通过在图像上绘制坐标轴或虚拟物体来实现。

## 函数解释
1. `cv2.findChessboardCorners()`：用于检测棋盘格的函数，返回值为bool值和角点坐标。
- 参数:
- `image`：输入图像。
- `patternSize`：棋盘格内角点的行列数。  
- 可选参数:  
- `cv2.CALIB_CB_ADAPTIVE_THRESH`：自适应阈值来处理图像，可以提高图像对比度，使角点更容易被检测到。  
- `cv2.CALIB_CB_NORMALIZE_IMAGE`：归一化图像来处理图像，可以避免图像亮度变化对角点检测的影响。  
- 返回值:  
- `corners`：检测到的角点坐标。
- `ret`：是否检测到棋盘格。 
2. `cv2.solvePnP()`：用于姿态估计的函数，返回值为旋转向量和平移向量。
- 参数:
- `objectPoints`：物体在空间中的坐标，即棋盘格角点的坐标。
- `imagePoints`：物体在图像中的坐标，即棋盘格角点的坐标。
- `cameraMatrix`：相机的内参数矩阵。
- `distCoeffs`：相机的畸变系数。
- 可选参数:
- `cv2.SOLVEPNP_ITERATIVE`：使用迭代算法进行姿态估计。
- `cv2.SOLVEPNP_P3P`：使用P3P算法进行姿态估计。
- `cv2.SOLVEPNP_EPNP`：使用EPNP算法进行姿态估计。
- `cv2.SOLVEPNP_DLS`：使用DLS算法进行姿态估计。
- `cv2.SOLVEPNP_UPNP`：使用UPNP算法进行姿态估计。
- 返回值:
- `rvec`：旋转向量。
- `tvec`：平移向量。
3. `cv2.projectPoints()`：用于将3D点投影到2D平面的函数，返回值为投影后的2D点坐标。
- 参数:
- `objectPoints`：物体在空间中的坐标，即棋盘格角点的坐标。
- `rvec`：旋转向量。
- `tvec`：平移向量。
- `cameraMatrix`：相机的内参数矩阵。
- `distCoeffs`：相机的畸变系数。
- 可选参数:
- `None`：不进行坐标轴的绘制。
- `axisPoints`：坐标轴的长度。
- 返回值:
- `imagePoints`：投影后的2D点坐标。





## 姿态估计画出坐标轴

In [10]:
import numpy as np
import cv2 as cv
import glob

# 画坐标轴
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())     
    img = cv.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)     
    img = cv.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img


# 加载先前保存的数据
with np.load('B.npz') as X: #加载内参矩阵,畸变系数,旋转向量,平移向量
    mtx, dist, _, _ = [X[i] for i in ('mtx','dist','rvecs','tvecs')]
#中止条件
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

#棋盘格角点的世界坐标
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

#负号表示它被拉向相机,因为棋盘格是在世界坐标系中定义的,所以所有Z坐标都为零
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
for fname in glob.glob('../data/left*.jpg'):
    img = cv.imread(fname)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray, (7,6),None)
    if ret == True:
        # 亚像素级角点检测(找到更精确的角点)
        corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        # 找到旋转和平移矢量。
        
        # solvePnP是求解空间中点的旋转和平移坐标与其在图像平面上的投影点之间的关系的问题。
        # solvePnP和外参矩阵的求解是一样的，只不过solvePnP是求解空间中点的旋转和平移坐标与其在图像平面上的投影点之间的关系的问题。
        ret,rvecs, tvecs = cv.solvePnP(objp, corners2, mtx, dist)
        
        # 将3D点投影到图像平面 3D点, 旋转向量, 平移向量, 内参矩阵, 畸变系数
        imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, mtx, dist)
        
        # 画出角点和坐标轴
        imgpts = np.int32(imgpts).reshape(-1,2)
        corners2 = np.int32(corners2).reshape(-1,2)
        
        img = draw(img,corners2,imgpts)
        cv.imshow('img',img)
        k = cv.waitKey(0) & 0xFF
        if k == ord('s'):
            cv.imwrite(fname[:6]+'.png', img)
cv.destroyAllWindows()


## 姿态估计画出立方体

In [9]:
import numpy as np
import cv2 as cv
import glob

# 画坐标轴
def draw(img, corners, imgpts):
    imgpts = np.int32(imgpts).reshape(-1,2)
    # 用绿色绘制底层
    img = cv.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)
    # 用蓝色绘制高
    for i,j in zip(range(4),range(4,8)):
        img = cv.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)
    # 用红色绘制顶层
    img = cv.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)
    return img

# 加载先前保存的数据
with np.load('B.npz') as X: #加载内参矩阵,畸变系数,旋转向量,平移向量
    mtx, dist, _, _ = [X[i] for i in ('mtx','dist','rvecs','tvecs')]
#中止条件
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

#棋盘格角点的世界坐标
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

#负号表示它被拉向相机,因为棋盘格是在世界坐标系中定义的,所以所有Z坐标都为零
axis = np.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0], [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3] ])

for fname in glob.glob('../data/left*.jpg'):
    img = cv.imread(fname)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray, (7,6),None)
    if ret == True:
        corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        # 找到旋转和平移矢量。
        ret,rvecs, tvecs = cv.solvePnP(objp, corners2, mtx, dist)
        # 将3D点投影到图像平面
        imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, mtx, dist)
        imgpts = np.int32(imgpts).reshape(-1,2)
        corners2 = np.int32(corners2).reshape(-1,2)
        
        img = draw(img,corners2,imgpts)
        cv.imshow('img',img)
        k = cv.waitKey(0) & 0xFF
        if k == ord('s'):
            cv.imwrite(fname[:6]+'.png', img)
cv.destroyAllWindows()
